# CNN

## 1. Relevant Principles

### 1.1 function of 1 * 1 convolution kernel

*卷积核*（convolutional kernel）：可以看作对某个局部的加权求和；它是对应局部感知，它的原理是在观察某个物体时我们既不能观察每个像素也不能一次观察整体，而是先从局部开始认识，这就对应了卷积。卷积核的大小一般有1x1,3x3和5x5的尺寸（一般是奇数x奇数）。

例如:
**输入: 224x224x3（rgb三通道)**
**输出: 32位深度，卷积核尺寸为5x5**

那么我们需要32个卷积核，每一个的尺寸为5x5x3（最后的3就是原图的rgb位深3），每一个卷积核的每一层是5x5（共3层）分别与原图的每层224x224卷积，然后将得到的三张新图叠加（算术求和），变成一张新的feature map。 每一个卷积核都这样操作，就可以得到32张新的feature map了。

不管输入图像的深度为多少，经过一个卷积核（filter），最后都通过下面的公式变成一个深度为1的特征图。不同的filter可以卷积得到不同的特征，也就是得到不同的feature map。
<img alt="image" height="400" src="./Image/cnn_kernel.png" width="400"/>

<img alt="image" height="100" src="./Image/size_fomula.gif" width="400"/>

*池化*（pooling）：卷积特征往往对应某个局部的特征。要得到global的特征需要将全局的特征执行一个aggregation（聚合）。池化就是这样一个操作，对于每个卷积通道，将更大尺寸（甚至是global）上的卷积特征进行pooling就可以得到更有全局性的特征。这里的pooling当然就对应了cross region。

***Answer: 与1x1的卷积相对应，而1x1卷积可以看作一个cross channel的pooling操作。pooling的另外一个作用就是升维或者降维，后面我们可以看到1x1的卷积也有相似的作用。***

当输入为6x6x32时，1x1卷积的形式是1x1x32，当只有一个1x1卷积核的时候，此时输出为6x6x1。此时便可以体会到1x1卷积的实质作用：降维。当1x1卷积核的个数小于输入channels数量时，即降维。
![image](./Image/size1_kernel.jpeg)

*Function*
跨通道信息交互（channal 的变换）

例子：使用1x1卷积核，实现降维和升维的操作其实就是channel间信息的线性组合变化，3x3，64channels的卷积核后面添加一个1x1，28channels的卷积核，就变成了3x3，28channels的卷积核，原来的64个channels就可以理解为跨通道线性组合变成了28channels，这就是通道间的信息交互。

### 1.2  parameter volume

**卷积层的参数量**: 参数量=（filter size * 前一层特征图的通道数 ）* 当前层filter数量
```
VGG-16为例，Conv1-1，输入 224*224*3，64个 3*3 filter，输出feature map 224 * 224 * 64 。

Conv1-1的参数量为 3*3*3*64 。

Conv2-1，输入 112 * 112 * 64 ，128个 3*3 filter，输出feature map 112*112*128 。

Conv2-1的参数量 3*3*64*128。
```

**全连接层的参数量**
```
VGG-16最后一次卷积得到的feature map为 [公式] ，全连接层是将feature map展开成一维向量 [公式] 。实际上，我们就是用4096个[公式] 的filter去做卷积（可以理解为是一个卷积层）。

计算第一个FC的参数量 7*7*512*4096 = 102760448

全连接层参数冗余。全连接层参数就可占整个网络参数80%左右，好吓人的。
```

**卷积层的计算量**

计算量 = 输出的feature map * 当前层filter
这仅仅是单个样本前向传播计算量，实际计算量还应乘以batch size。

```
VGG-16最后一次卷积得到的feature map为 [公式] ，全连接层是将feature map展开成一维向量 [公式] 。则FC层的计算量为[公式] 。
```
![image](./Image/vgg16.jpg)


### 1.3 convolution methods

1. DepthWise Convolution**  (Xception MobileNet)
```
我们首先对每一个通道进行各自的卷积操作，有多少个通道就有多少个过滤器。得到新的通道feature maps之后，这时再对这批新的通道feature maps进行标准的1×1跨通道卷积操作。这种操作被称为“DepthWise convolution”，缩写“DW”。
```

```
假设输入通道数为3，要求输出通道数为256

method 1(nomal convolution): 直接接一个3×3×256的卷积核，参数量为：3×3×3×256 = 6,912
method 2(DW convolution): 分两步完成，那么depthwise separable convolution就是先用M个3*3卷积核一对一卷积输入的M个feature map，不求和，生成M个结果；然后用N个1*1的卷积核正常卷积前面生成的M个结果，求和，最后生成N个结果. 参数量为：3×3×3 + 3×1×1×256 = 795，又把参数量降低到九分之一！
```

2. 空洞卷积（Dilated convolution）
![image](./Image/dilated_kernel.jpeg)

```
空洞卷积：在3*3卷积核中间填充0，有两种实现方式，第一，卷积核填充0，第二，输入等间隔采样。
function:
1. 扩大感受野:
在deep net中为了增加感受野且降低计算量，总要进行降采样(pooling或s2/conv)，这样虽然可以增加感受野，但空间分辨率降低了。为了能不丢失分辨率，且仍然扩大感受野，可以使用空洞卷积。这在检测，分割任务中十分有用。一方面感受野大了可以检测分割大目标，另一方面分辨率高了可以精确定位目标。
2. 捕获多尺度上下文信息：
空洞卷积有一个参数可以设置dilation rate，具体含义就是在卷积核中填充dilation rate-1个0，因此，当设置不同dilation rate时，感受野就会不一样，也即获取了多尺度信息。多尺度信息在视觉任务中相当重要啊。
```
### 1.4 dropout
Dropout可以作为训练深度神经网络的一种trick供选择。在每个训练批次中，通过忽略一半的特征检测器（让一半的隐层节点值为0），可以明显地减少过拟合现象。这种方式可以减少特征检测器（隐层节点）间的相互作用，检测器相互作用是指某些检测器依赖其他检测器才能发挥作用。

Dropout说的简单一点就是：我们在前向传播的时候，让某个神经元的激活值以一定的概率p停止工作，这样可以使模型泛化性更强，因为它不会太依赖某些局部的特征。
![image](./Image/dropout.jpg)

```
step 1: 首先随机（临时）删掉网络中一半的隐藏神经元，输入输出神经元保持不变（图3中虚线为部分临时被删除的神经元）
step 2: 然后把输入x通过修改后的网络前向传播，然后把得到的损失结果通过修改的网络反向传播。一小批训练样本执行完这个过程后，在没有被删除的神经元上按照随机梯度下降法更新对应的参数（w，b）.
step 3: 然后继续重复这一过程：
        * 恢复被删掉的神经元（此时被删除的神经元保持原样，而没有被删除的神经元已经有所更新）
        * 从隐藏层神经元中随机选择一个一半大小的子集临时删除掉（备份被删除神经元的参数）。
        * 对一小批训练样本，先前向传播然后反向传播损失并根据随机梯度下降法更新参数（w，b） （没有被删除的那一部分参数得到更新，删除的神经元参数保持被删除前的结果）。
不断重复这一过程。
```

```
（1）取平均的作用:
        先回到标准的模型即没有dropout，我们用相同的训练数据去训练5个不同的神经网络，一般会得到5个不同的结果，此时我们可以采用 “5个结果取均值”或者“多数取胜的投票策略”去决定最终结果。例如3个网络判断结果为数字9,那么很有可能真正的结果就是数字9，其它两个网络给出了错误结果。这种“综合起来取平均”的策略通常可以有效防止过拟合问题。因为不同的网络可能产生不同的过拟合，取平均则有可能让一些“相反的”拟合互相抵消。dropout掉不同的隐藏神经元就类似在训练不同的网络，随机删掉一半隐藏神经元导致网络结构已经不同，整个dropout过程就相当于对很多个不同的神经网络取平均。而不同的网络产生不同的过拟合，一些互为“反向”的拟合相互抵消就可以达到整体上减少过拟合。
（2）减少神经元之间复杂的共适应关系:
        因为dropout程序导致两个神经元不一定每次都在一个dropout网络中出现。这样权值的更新不再依赖于有固定关系的隐含节点的共同作用，阻止了某些特征仅仅在其它特定特征下才有效果的情况 。迫使网络去学习更加鲁棒的特征 ，这些特征在其它的神经元的随机子集中也存在。换句话说假如我们的神经网络是在做出某种预测，它不应该对一些特定的线索片段太过敏感，即使丢失特定的线索，它也应该可以从众多其它线索中学习一些共同的特征。从这个角度看dropout就有点像L1，L2正则，减少权重使得网络对丢失特定神经元连接的鲁棒性提高。
```